## 4. Data Operations: Grouping, Aggregation, and Shuffling

Let's look at some more involved transformations.

#### Custom batching using **`groupby`** 

In case you want to generate batches according to a **specific key**, you can use **`groupby` to group the data** by the key and then use **`map_groups`** to apply the transformation.

For instance, let's compute the accuracy of the model by **"ground truth label"**.

In [23]:
def add_label(batch: dict[str, np.ndarray]) -> dict[str, np.ndarray]:
    batch["ground_truth_label"] = [int(path.split("/")[-2]) for path in batch["path"]]
    return batch


def compute_accuracy(group: dict[str, np.ndarray]) -> dict[str, np.ndarray]:
    return {
        "accuracy": [np.mean(group["predicted_label"] == group["ground_truth_label"])],
        "ground_truth_label": group["ground_truth_label"][:1],
    }


ds_preds.map_batches(add_label).groupby("ground_truth_label").map_groups(compute_accuracy).to_pandas()

2024-12-06 09:59:09,827	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-06_07-31-15_546129_2446/logs/ray-data
2024-12-06 09:59:09,828	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ExpandPaths] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(normalize)] -> ActorPoolMapOperator[MapBatches(MNISTClassifier)] -> TaskPoolMapOperator[MapBatches(add_label)] -> AllToAllOperator[Sort] -> TaskPoolMapOperator[MapBatches(compute_accuracy)]


- ExpandPaths 1: 0 bundle [00:00, ? bundle/s]

- ReadFiles 2: 0 bundle [00:00, ? bundle/s]

- MapBatches(normalize) 3: 0 bundle [00:00, ? bundle/s]

- MapBatches(MNISTClassifier) 4: 0 bundle [00:00, ? bundle/s]

...

Shuffle Reduce 9:   0%|          | 0/1 [00:00<?, ? bundle/s]

- MapBatches(compute_accuracy) 10: 0 bundle [00:00, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

Sort Sample 0:   0%|          | 0/1 [00:00<?, ? block/s]

,accuracy,ground_truth_label
0,0.98,0
1,1.00,1
2,1.00,2
3,0.96,3
4,1.00,4
5,1.00,5
6,0.98,6
7,0.98,7
8,1.00,8
9,0.98,9


<div class="alert alert-block alert-warning">

<b>Note:</b> ds_preds is not re-computed given we have already materialized the dataset.

</div>

### Aggregations

Ray Data also supports a **variety of aggregations**. These aggregations can be **chained**.

For instance, we can compute the **mean** accuracy across the entire dataset.

In [24]:
ds_preds.map_batches(add_label).map_batches(compute_accuracy).mean(on="accuracy")

2024-12-06 10:01:43,102	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-06_07-31-15_546129_2446/logs/ray-data
2024-12-06 10:01:43,103	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ExpandPaths] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(normalize)] -> ActorPoolMapOperator[MapBatches(MNISTClassifier)] -> TaskPoolMapOperator[MapBatches(add_label)->MapBatches(compute_accuracy)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


- ExpandPaths 1: 0 bundle [00:00, ? bundle/s]

- ReadFiles 2: 0 bundle [00:00, ? bundle/s]

- MapBatches(normalize) 3: 0 bundle [00:00, ? bundle/s]

- MapBatches(MNISTClassifier) 4: 0 bundle [00:00, ? bundle/s]

...

Shuffle Map 7:   0%|          | 0/1 [00:00<?, ? bundle/s]

Shuffle Reduce 8:   0%|          | 0/1 [00:00<?, ? bundle/s]

- limit=1 9: 0 bundle [00:00, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

0.988

As of version 2.34.0, Ray Data provides the following aggregation functions:

- `count`
- `max`
- `mean`
- `min`
- `sum`
- `std`

See relevant [docs page here](https://docs.ray.io/en/latest/data/api/grouped_data.html#ray.data.aggregate.AggregateFn).

### Shuffling data 

There are **different options to shuffle data** in Ray Data of varying degrees of randomness and performance.

#### File based shuffle on read

To randomly **shuffle the ordering of input files before reading**, call a read function that supports shuffling, such as `read_images()`, and use the shuffle="files" parameter.

In [25]:
ray.data.read_images("s3://anyscale-public-materials/ray-ai-libraries/mnist/50_per_index/", shuffle="files")

2024-12-06 10:04:12,645	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-06_07-31-15_546129_2446/logs/ray-data
2024-12-06 10:04:12,645	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage]


- ReadImage 1: 0 bundle [00:00, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

2024-12-06 10:04:13,055	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-06_07-31-15_546129_2446/logs/ray-data
2024-12-06 10:04:13,056	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage]


- ReadImage 1: 0 bundle [00:00, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

Dataset(
   num_rows=500,
   schema={image: numpy.ndarray(shape=(28, 28), dtype=uint8)}
)

#### Shuffling block order

- This option randomizes the **order of blocks in a dataset**. 
- Blocks are the **basic unit** of data chunk that Ray Data stores in the object store. 
- Applying this operation alone **doesn’t involve heavy computation and communication**. 
- However, it requires Ray Data to **materialize all blocks in memory** before applying the operation. 
- Only use this option when your dataset is small enough to fit into the object store memory.

To perform block order shuffling, use `randomize_block_order`.

In [26]:
ds_randomized_blocks = ds_preds.randomize_block_order()
ds_randomized_blocks.materialize()

2024-12-06 10:05:33,949	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-06_07-31-15_546129_2446/logs/ray-data
2024-12-06 10:05:33,950	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ExpandPaths] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(normalize)] -> ActorPoolMapOperator[MapBatches(MNISTClassifier)] -> AllToAllOperator[RandomizeBlockOrder]


- ExpandPaths 1: 0 bundle [00:00, ? bundle/s]

- ReadFiles 2: 0 bundle [00:00, ? bundle/s]

- MapBatches(normalize) 3: 0 bundle [00:00, ? bundle/s]

- MapBatches(MNISTClassifier) 4: 0 bundle [00:00, ? bundle/s]

- RandomizeBlockOrder 5: 0 bundle [00:00, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

MaterializedDataset(
   num_blocks=1,
   num_rows=500,
   schema={
      image: numpy.ndarray(shape=(1, 28, 28), dtype=float),
      path: string,
      predicted_label: int64
   }
)

#### Shuffle rows globally

- To randomly **shuffle all rows globally**, call `random_shuffle()`. 
- This is the **slowest option** for shuffle, and requires **transferring data across network between workers**. 
- This option achieves the **best randomness** among all options.


In [27]:
ds_randomized_rows = ds_preds.random_shuffle()
ds_randomized_rows.materialize()

2024-12-06 10:06:32,993	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-06_07-31-15_546129_2446/logs/ray-data
2024-12-06 10:06:32,993	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ExpandPaths] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(normalize)] -> ActorPoolMapOperator[MapBatches(MNISTClassifier)] -> AllToAllOperator[RandomShuffle]


- ExpandPaths 1: 0 bundle [00:00, ? bundle/s]

- ReadFiles 2: 0 bundle [00:00, ? bundle/s]

- MapBatches(normalize) 3: 0 bundle [00:00, ? bundle/s]

- MapBatches(MNISTClassifier) 4: 0 bundle [00:00, ? bundle/s]

- RandomShuffle 5: 0 bundle [00:00, ? bundle/s]

Shuffle Map 6:   0%|          | 0/1 [00:00<?, ? bundle/s]

Shuffle Reduce 7:   0%|          | 0/1 [00:00<?, ? bundle/s]

Running 0: 0 bundle [00:00, ? bundle/s]

MaterializedDataset(
   num_blocks=1,
   num_rows=500,
   schema={
      image: numpy.ndarray(shape=(1, 28, 28), dtype=float),
      path: string,
      predicted_label: int64
   }
)